## Clasificación TF-IDF 

In [19]:
import pandas as pd
from gensim import corpora, models, similarities
from gensim.utils import simple_preprocess

Lo primero que necesitamos es cargar nuestros glosarios de términos para crear nuestro diccionario

In [20]:
def cargar_glosario(categoria, conjunto):
    fname = f"../Datos/Glosarios/{conjunto}/glosario_{categoria}.txt"

    glosario = []
    with open(fname, 'r') as f:
        glosario = [termino.rstrip('\n') for termino in f.readlines()]
    return glosario

In [21]:
categorias = ["deportes", "salud", "ciencia", "politica"]

glosarios = {}
for categoria in categorias:
    glosarios[categoria] = cargar_glosario(categoria, "train")

print(glosarios)

{'deportes': ['falso', 'jamas', 'smash', 'mbappe', 'gigante', 'reserva', 'nets', 'carpena', 'djokovic', 'mclaren', 'exencion', 'butler', 'cuento', 'formato', 'campazzo', 'tenerife', 'domenicali', 'booker', 'tatum', 'madrid', 'real', 'premio', 'juventus', 'pagar', 'ronaldo', 'magnussen', 'steiner', 'gavi', 'porra', 'indycar', 'palou', 'gonzalez', 'gonzalo', 'rotacion', 'florentino', 'heredero', 'verdasco', 'enrique', 'luis', 'cristiano', 'cd', 'marbella', 'residencia', 'serbia', 'serbio', 'rosa', 'collins', 'jimmy', 'arabia', 'club'], 'salud': ['arterial', 'dash', 'anemia', 'folcodina', 'cafa', 'genital', 'vrs', 'unicef', 'lobo', 'verruga', 'aceite', 'mascarilla', 'pulmonar', 'presion', 'cannabis', 'pet', 'epilepsia', 'zumo', 'quiraorgica', 'afiliado', 'eps', 'ips', 'tc', 'creatividad', 'condiloma', 'marfan', 'manada', 'parasito', 'lavar', 'coosalud'], 'ciencia': ['vacao', 'llama', 'captura', 'cola', 'honor', 'gemanidas', 'banyoles', 'neandertal', 'agujero', 'latigo', 'supersa', 'perfec

Creamos nuestro diccionario de palabras en base a los terminos de todos los glosarios

In [22]:
def create_dictionary(glosarios):
    doc_tokens = [[termino for termino in glosario] for glosario in glosarios.values()]
    dictionary = corpora.Dictionary(doc_tokens)
    return dictionary

In [23]:
glosarios_dict = create_dictionary(glosarios)
print(glosarios_dict)

Dictionary<140 unique tokens: ['arabia', 'booker', 'butler', 'campazzo', 'carpena']...>


Cargamos nuestras noticias de test y las convertimos a un bag of words utilizando nuestro diccionario

In [24]:
noticias_test_dataframe = pd.read_csv("../Datos/noticias_test.csv")

In [25]:
def create_bag_of_words(docs_list, dictionary):
    doc_tokens = [simple_preprocess(corpus) for corpus in docs_list]
    docs_bow = [dictionary.doc2bow(doc) for doc in doc_tokens]
    return docs_bow

In [26]:
def create_tfidf(docs_bow):
    docs_tfidf = models.TfidfModel(docs_bow, smartirs="lfc")
    return docs_tfidf

Ahora es momento de crear una bag of words para cada noticia en base a nuestro diccionario

In [27]:
docs_bow = create_bag_of_words(noticias_test_dataframe["corpus"].values, glosarios_dict)
print(docs_bow[19])

[(13, 8), (49, 5), (133, 2)]


Y con el bag of words podemos crear nuestro modelo tfidf

In [28]:
tfidf_model = create_tfidf(docs_bow)
tfidf_model[docs_bow[0]]

[(1, 0.9809846216503764), (102, 0.1940854762352605)]

Representamos nuestros glosarios en tfidf

In [29]:
deportes_bow = glosarios_dict.doc2bow(glosarios["deportes"])
deportes_tfidf = tfidf_model[deportes_bow]

salud_bow = glosarios_dict.doc2bow(glosarios["salud"])
salud_tfidf = tfidf_model[salud_bow]

ciencia_bow = glosarios_dict.doc2bow(glosarios["ciencia"])
ciencia_tfidf = tfidf_model[ciencia_bow]

politica_bow = glosarios_dict.doc2bow(glosarios["politica"])
politica_tfidf = tfidf_model[politica_bow]

In [30]:
print(deportes_bow)

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1)]


Ahora clasificamos

In [31]:
index = similarities.MatrixSimilarity(tfidf_model[docs_bow])
similarities_deportes = index[deportes_tfidf]
similarities_salud = index[salud_tfidf]
similarities_ciencia = index[ciencia_tfidf]
similarities_politica = index[politica_tfidf]

In [32]:
def get_prediction(index, 
                   similarities_deportes,       
                   similarities_salud,                       
                   similarities_ciencia,
                   similarities_politica):
    mejor_match = 0
    mejor = "No clasificado"
    if (similarities_deportes[index] > mejor_match):
        mejor_match = similarities_deportes[index]
        mejor = "sports"
    if (similarities_salud[index] > mejor_match):
        mejor_match = similarities_salud[index]
        mejor = "health"
    if (similarities_ciencia[index] > mejor_match):
        mejor_match = similarities_ciencia[index]
        mejor = "science"
    if (similarities_politica[index] > mejor_match):
        mejor_match = similarities_politica[index]
        mejor = "politics"
    return mejor
    

In [33]:
predictions = []
for i in range(len(docs_bow)):
    pred = get_prediction(i, 
                          similarities_deportes,
                          similarities_salud,
                          similarities_ciencia,
                          similarities_politica)
    predictions.append(pred)

In [34]:
noticias_test_dataframe.insert(2, "labels", predictions)

In [35]:
noticias_test_dataframe.to_csv('../Prediccion/predicciones.csv')

In [36]:
display(noticias_test_dataframe)

,index,category,labels,n_doc,title,path,link,docs,corpus
0,21,sports,sports,22,Houston da la sorpresa remontando y derrotando...,./Datos/Raw_data/sports/22.txt,https://news.google.com/__i/rss/rd/articles/CB...,Jalen Green y los jóvenes Rockets sonrieron an...,Jalen Green joven Rockets sonreir anoche terri...
1,5,sports,sports,6,"Cristiano Ronaldo exige a la Juventus 19,9 mil...",./Datos/Raw_data/sports/6.txt,https://news.google.com/__i/rss/rd/articles/CB...,El portugués reclama al club italiano los sala...,portugues reclamar club italiano salario pacta...
2,48,sports,sports,49,Alpine comenzó 2022 “en ruinas” - AS,./Datos/Raw_data/sports/49.txt,https://news.google.com/__i/rss/rd/articles/CB...,Alpine llegaba con grandes esperanzas a 2022. ...,Alpine llegar esperanza 2022 expectacion maxim...
3,13,sports,sports,14,Boicot de Mbappé al alcohol - AS,./Datos/Raw_data/sports/14.txt,https://news.google.com/__i/rss/rd/articles/CB...,"Desde que comenzó el Mundial, Kylian Mbappé ha...",comenzar Mundial Kylian Mbappe seguir patron c...
4,46,sports,science,47,KTM recibe ayuda de la F1 - AS,./Datos/Raw_data/sports/47.txt,https://news.google.com/__i/rss/rd/articles/CB...,"El techo presupuestario de la F1, tan polémico...",techo presupuestario F1 polemico temporada exc...
...,...,...,...,...,...,...,...,...,...
115,198,politics,politics,19,El PSOE se queda solo y no consigue limitar la...,./Datos/Raw_data/politics/19.txt,https://elpais.com/america-colombia/2022-12-08...,Castillo durÃ³ apenas un aÃ±o y medio en el po...,Castillo dura3 aao inestabilidad constante cen...
116,216,politics,health,37,Petro asegura que Pedro Castillo “se dejó llev...,./Datos/Raw_data/politics/37.txt,https://www.20minutos.es/noticia/5084030/0/rib...,Traducido al mensaje que tambiÃ©n la ComisiÃ³n...,traducido mensaje tambiA ComisiA3n Europea -qu...
117,221,politics,politics,42,Meloni cancela su viaje a Alicante por unagrip...,./Datos/Raw_data/politics/42.txt,https://elpais.com/espana/2022-12-13/bolanos-s...,"Por otro lado, el presidente del grupo parlame...",presidente grupo parlamentario Unidas Jaume As...
118,195,politics,politics,16,El número de hogares acogidos a la tarifaregul...,./Datos/Raw_data/politics/16.txt,https://elpais.com/ideas/2022-12-09/la-subjeti...,Porque ambos exabruptos tienen defensa subjeti...,exabrupto defensa subjetivo partidario cultura...
